In [1]:
"""
Connect senior:Mainland
"""
import pandas as pd
import json
import requests
def ssql(sql):
    admin_token = '2cebeb5c7edac5ef0d7c5a640e69fc7d43bf2cd6a24ce43dfee3dd33662c0bab'
    url = 'http://sensor.wb-intra.com/api/sql/query?token=%s&project=production' % admin_token
    data = {'q': sql, 'format': 'json'}
    req = requests.post(url,data)
    req_dec = req.content.decode()
    try:
        req_json = json.loads('[' + req_dec.replace('\n', ',')[:-1] + ']')
        df_d_id = pd.DataFrame(req_json)
        return df_d_id
    except:
        print(req_dec)

In [3]:
df = ssql("""
SELECT t1.date,
       t1.distinct_id,
       t1.gameSubType,
       t1.game_num/t2.ready_num as avg_ready
FROM
  (SELECT date,distinct_id,
               gameSubType,
               count(distinct_id)AS game_num
   FROM events
   WHERE event = 'gameStart'
     AND gameTypeId = 1800
     AND date BETWEEN '2020-12-16' AND current_date()
     AND time >= '2020-12-16 12:59:59'
   GROUP BY 1,
            2,
            3) t1
RIGHT JOIN
  (SELECT date,distinct_id,
               gameSubType,
               count(distinct_id)AS ready_num
   FROM events
   WHERE event = 'action'
     AND gameTypeId = 1800
     AND op_type = 'game_ready'
     AND time >= '2020-12-16 12:59:59'
     AND date BETWEEN '2020-12-16' AND current_date()
   GROUP BY 1,
            2,
            3)t2 ON t1.distinct_id = t2.distinct_id
AND t1.date = t2.date""")

In [4]:
df

,date,distinct_id,gamesubtype,avg_ready
0,2020-12-16,141159328,1,0.462
1,2020-12-16,123416184,1,0.500
2,2020-12-16,137252934,3,0.889
3,2020-12-16,141467200,1,11.000
4,2020-12-16,141467200,2,1.000
...,...,...,...,...
53762,2020-12-16,147611814,2,0.708
53763,2020-12-16,122667869,1,0.400
53764,2020-12-16,118949402,1,0.429
53765,2020-12-16,110569481,1,0.750


In [5]:
df.groupby(['gamesubtype'])['avg_ready'].describe()

,count,mean,std,min,25%,50%,75%,max
gamesubtype,,,,,,,,
1,20090.0,0.947529,1.494706,0.020,0.5,0.667,1.0,25.0
2,8194.0,1.025953,1.653261,0.024,0.5,0.714,1.0,32.0
3,12379.0,0.879417,1.163580,0.021,0.5,0.667,1.0,21.0
4,12726.0,0.786529,1.010868,0.020,0.4,0.571,1.0,16.0


In [12]:
df = ssql("""
SELECT t1.distinct_id
FROM
  (SELECT distinct_id
   FROM events
   WHERE event = 'gameStart'
     AND gameTypeId = 1800
     AND date = '2020-11-30'
   GROUP BY 1) t1
LEFT JOIN
  (SELECT distinct_id
   FROM events
   WHERE event = 'login'
     AND appId IN ('20014',
                   '30015',
                   '10001',
                   '10002')
     AND date >= '2020-12-01'
   GROUP BY 1)t2 ON t1.distinct_id = t2.distinct_id
WHERE t2.distinct_id IS NULL

""")

In [13]:

"""
Connect Aliyun and commit MySQL
"""
 
import requests
import json
 
 
def aliq(db, req):
    url = 'http://localhost:8008/aliq?sql=%s&db=%s' % (requests.utils.quote(req), db)
    resp = requests.get(url)
    res = resp.content.decode()
    js = json.loads(res)
    return js

In [14]:
wolf_user_list = list(df.distinct_id)
group_user_dict = {str(i) : [] for i in range(100)}
for i in wolf_user_list:
    no = str(int(i[-2:]))
    group_user_dict[no].append(i)
 
all_info = []
for i in range(100):   
    if group_user_dict[str(i)]:
        table = 'user_phone_%s' % str(i)
 
        user_list = group_user_dict[str(i)]
        if len(user_list) == 1:
            user_list.append('123')
        sql = """
            select uid, phone_number
            from {table}
            where uid in{user_list}
        """.format(table=table, user_list=tuple(user_list))
         
        data = aliq('spy', sql)
        if data[0]:
            all_info.extend(data)
df_phone = pd.DataFrame.from_dict([{'distinct_id': str(i[0]), 'phone_number': str(i[1])} for i in all_info])
df_merge = pd.merge(df, df_phone, on='distinct_id', how='left')
df_merge

,distinct_id,phone_number
0,114012710,8618531450124
1,142963132,NaN
2,114686800,NaN
3,124930823,8618838591869
4,122994209,8615356002492
...,...,...
47907,107194565,8618701152907
47908,112346619,8615589772981
47909,139752197,8613722506264
47910,126329019,8619931048778


In [15]:
df_merge.to_clipboard()